In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("Mcomp")
install.packages("forecast")
install.packages("data.table")
install.packages("magrittr")
install.packages("stringr")
install.packages("foreach")

library(Mcomp)
library(forecast)
packageVersion("forecast")
library(data.table)
library(magrittr)
library(stringr)
library(foreach)

In [ ]:
%%R
M4M_actuals_train <- fread('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Train/Monthly-train.csv', header = TRUE, fill = TRUE)
M4M_actuals_test <- fread('https://raw.githubusercontent.com/Mcompetitions/M4-methods/master/Dataset/Test/Monthly-test.csv', header = TRUE, fill = TRUE)

setnames(M4M_actuals_train, names(M4M_actuals_train), c('item_id', 1:2794))
setnames(M4M_actuals_test, names(M4M_actuals_test), c('item_id', 1:18))
M4M_actuals_train[, item_id := str_remove(item_id, 'M') %>% as.numeric()]
M4M_actuals_test[, item_id := str_remove(item_id, 'M') %>% as.numeric()]

M4M_actuals_all <- merge.data.table(M4M_actuals_train, M4M_actuals_test, by = 'item_id')
setnames(M4M_actuals_all, names(M4M_actuals_all), c('item_id', 1:2812))
M4M_actuals_all <- melt.data.table(M4M_actuals_all, id.vars = 'item_id')
M4M_actuals_all <- M4M_actuals_all[!is.na(value)]
setorder(M4M_actuals_all, variable)
M4M_actuals_all[, variable:= c(1:.N) , by = list(item_id)]

In [ ]:
%%R
start_item <- 9001
end_item <- 10000

In [ ]:
%%R
M4M_actuals_all_df <- data.frame(M4M_actuals_all)

starttime <- Sys.time()

ets_cforecasts_df <- foreach(id = c(start_item:end_item), .combine = 'rbind') %do% {

  ets_forecasts_cmean_df <- data.frame()
  ets_forecasts_csd_df <- data.frame()

  ts <- M4M_actuals_all_df[which(M4M_actuals_all_df$item_id == id),]$value

    counter <- 1
    for (fc_origin in 1:13){

      ts_origin <- ts(ts[1:(length(ts)-(18-(fc_origin-1)))], frequency = 12)

      ets_fit <- ets(ts_origin)

      nsim <- 1000
      sim <- matrix(nrow = nsim, ncol = 6)
      for(i in seq_len(nsim)){
        sim_i <- simulate(ets_fit, future = TRUE, nsim = 6)
        sim[i,1] <- sum(sim_i[1])
        sim[i,2] <- sum(sim_i[1:2])
        sim[i,3] <- sum(sim_i[1:3])
        sim[i,4] <- sum(sim_i[1:4])
        sim[i,5] <- sum(sim_i[1:5])
        sim[i,6] <- sum(sim_i[1:6])
      }

      sim_dt <- data.table(sim)
      mean_agg <- unlist(sim_dt[, lapply(.SD, mean)])
      sd_agg <- unlist(sim_dt[, lapply(.SD, sd)])

      ets_forecasts_cmean_df[counter, 1] <- id
      ets_forecasts_cmean_df[counter, 2] <- fc_origin
      ets_forecasts_cmean_df[counter, c(3:8)] <- mean_agg
      ets_forecasts_cmean_df[counter, 9] <- 'cmean_forecast'

      ets_forecasts_csd_df[counter, 1] <- id
      ets_forecasts_csd_df[counter, 2] <- fc_origin
      ets_forecasts_csd_df[counter, c(3:8)] <- sd_agg
      ets_forecasts_csd_df[counter, 9] <- 'csd_forecast'

      counter <- counter+1

    }

    #output
    rbind(ets_forecasts_cmean_df, ets_forecasts_csd_df)

}

print(Sys.time() - starttime)

colnames(ets_cforecasts_df) <- c('item_id', 'fc_origin', as.character(1:6), 'type')
ets_cforecasts_dt <- ets_cforecasts_df %>% data.table()

In [ ]:
%%R
fwrite(ets_cforecasts_dt, paste0('/content/drive/My Drive/Colab Notebooks/Sweeps2/M4_ETS_cumul_probabilistic_',start_item,'to',end_item,'.csv'))